# Data Preprocessing

The original data consists of three parts, drug prices data from 2017 to 2020, drug prices data from 2014 to 2016 and drug details data from IQVIA. The original data structures for data after 2017 and before 2017 are different, so the processings are different either.

In [1]:
# Importing libraries

import pandas as pd
from datetime import datetime
import numpy as np
import zipfile as z
from tqdm import tqdm

# Part 1 - Merging all data

### Merging 2017-2020 prices data with drug details data

In [2]:
## We have several zip files and each zip file contains that year's price data 
## Merge all data in each zip file to one zip file
zips = [str(i)+'_prices.zip' for i in range(2017,2021)]

"""
Open the first zip file as append and then read all
subsequent zip files and append to the first one
"""
with z.ZipFile(zips[0], 'a') as z1:
    for fname in zips[1:]:
        zf = z.ZipFile(fname, 'r')
        for n in zf.namelist():
            z1.writestr(n, zf.open(n).read())
            
pricingData = z.ZipFile('2017_prices.zip')


In [ ]:
## IQVIA data contains drug details
iqvia = 'IQVIA Additional Drug Detail v72020.xlsx'
pricingData = z.ZipFile('2017_prices.zip')
df_IQVIA_Data = pd.read_excel(iqvia)

In [ ]:
# It's important to format the drug NDCs to include 0s, especially true for 
# NDCs that have leading 0's.  Often these are dropped because the files are 
# loaded from CSVs.

df_IQVIA_Data['NDC'] = [e[0] for e in df_IQVIA_Data.NDC.str.split(' ')]
df_IQVIA_Data.NDC.astype(str).str.zfill(11) # Fill 0 from beginning until the length is 11
df_IQVIA_Data.head()

In [ ]:
'''
The following loops over all CSVs in a zip file and assigns a Date column with the date 
portion of the file name 
'''

price_dates = []

for f in pricingData.namelist():
    if f.split('_')[2][6:] == '01':
        df = pd.read_csv(pricingData.open(f))
        
        # Since we are only focusing on Brand, I filter out the generics.
        df = df.loc[df['Brand/Generic'] == 'Brand']
        
        # Each csv file contains the date, I strip out this field and assign it to a Date column
        df['Month'] = datetime.strptime(f.split('_')[2],'%Y%m%d').month
        df['Year'] = datetime.strptime(f.split('_')[2],'%Y%m%d').year
        df = df[['Drug Identifier','Drug Group','Brand/Generic','Manufacturer','WAC','Month','Year']]
        df['Drug Identifier'] = df['Drug Identifier'].astype(str).str.zfill(11)
        price_dates.append(df)
    
df_Pricing_Data = pd.concat(price_dates, ignore_index=True)

In [ ]:
'''IQVIA Data is used for left merge so that result consists of complete drug data (price + detail).  
'''
merge_ = pd.merge(df_IQVIA_Data, df_Pricing_Data, left_on='NDC', right_on='Drug Identifier', how='inner')

In [ ]:
# Pivot
# We have several columns for the volume per month, say 'Aug 2014\nTRx', 'Sep 2014\nTRx',etc. 
# Here we are creating a new column indicate that month's Trx volume
columns = ['Aug 2014\nTRx', 'Sep 2014\nTRx', 'Oct 2014\nTRx',
       'Nov 2014\nTRx', 'Dec 2014\nTRx', 'Jan 2015\nTRx', 'Feb 2015\nTRx',
       'Mar 2015\nTRx', 'Apr 2015\nTRx', 'May 2015\nTRx', 'Jun 2015\nTRx',
       'Jul 2015\nTRx', 'Aug 2015\nTRx', 'Sep 2015\nTRx', 'Oct 2015\nTRx',
       'Nov 2015\nTRx', 'Dec 2015\nTRx', 'Jan 2016\nTRx', 'Feb 2016\nTRx',
       'Mar 2016\nTRx', 'Apr 2016\nTRx', 'May 2016\nTRx', 'Jun 2016\nTRx',
       'Jul 2016\nTRx', 'Aug 2016\nTRx', 'Sep 2016\nTRx', 'Oct 2016\nTRx',
       'Nov 2016\nTRx', 'Dec 2016\nTRx', 'Jan 2017\nTRx', 'Feb 2017\nTRx',
       'Mar 2017\nTRx', 'Apr 2017\nTRx', 'May 2017\nTRx', 'Jun 2017\nTRx',
       'Jul 2017\nTRx', 'Aug 2017\nTRx', 'Sep 2017\nTRx', 'Oct 2017\nTRx',
       'Nov 2017\nTRx', 'Dec 2017\nTRx', 'Jan 2018\nTRx', 'Feb 2018\nTRx',
       'Mar 2018\nTRx', 'Apr 2018\nTRx', 'May 2018\nTRx', 'Jun 2018\nTRx',
       'Jul 2018\nTRx', 'Aug 2018\nTRx', 'Sep 2018\nTRx', 'Oct 2018\nTRx',
       'Nov 2018\nTRx', 'Dec 2018\nTRx', 'Jan 2019\nTRx', 'Feb 2019\nTRx',
       'Mar 2019\nTRx', 'Apr 2019\nTRx', 'May 2019\nTRx', 'Jun 2019\nTRx',
       'Jul 2019\nTRx', 'Aug 2019\nTRx', 'Sep 2019\nTRx', 'Oct 2019\nTRx',
       'Nov 2019\nTRx', 'Dec 2019\nTRx', 'Jan 2020\nTRx', 'Feb 2020\nTRx',
       'Mar 2020\nTRx', 'Apr 2020\nTRx', 'May 2020\nTRx', 'Jun 2020\nTRx',
       'Jul 2020\nTRx']

temp = merge_.melt(id_vars=[ 'NDC','Year','Month'],  value_vars=columns)
temp.value.fillna(0,inplace = True) ## Fill NA with 0, since NA means the drug was not dispensed that month
dict_={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

def fun(year, month, column):
    if dict_[month] == column[:3] and year == int(column[4:8]):
        return True
    else:
        return False
    
temp['a'] = temp.apply(lambda x:fun(x.Year,x.Month,x.variable), axis = 1)
temp = temp[temp.a == True]
temp.drop(['a','variable'],axis = 1,inplace=True)
temp.rename(columns={"value": "TRx"},inplace = True)
merge_ =  pd.merge(merge_, temp,  how='inner', left_on=['NDC','Year','Month'], right_on = ['NDC','Year','Month'])
merge_ = merge_.loc[:, ~merge_.columns.isin(columns)]

### Merging with 2014-2016 drug prices data

In [ ]:
# For drug prices data, it has different format 
# Different processing is operated
df = pd.read_excel('2014-2016 WAC History.xlsx')

df = pd.concat([pd.DataFrame({'Date': pd.date_range(row.EffectiveDate, row.EndDate, freq='M'),
                              'WAC': row.Price,
                              'Drug Identifier': row.Identifier}, 
                             columns=['Date', 'WAC', 'Drug Identifier']) 
               for i, row in df.iterrows()], ignore_index=True)

df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.apply(lambda x:x.month)
df['Year'] = df.Date.apply(lambda x:x.year)
df.drop('Date',axis=1,inplace = True)
df.rename(columns={"Drug Identifier": "NDC"}, inplace = True)

# Merging 2017-2020 data with 2014-2016
df = pd.concat([df, merge_],sort = True)
df.drop_duplicates(keep = 'first',inplace = True)

# Part 2 - Adding percent change and some other useful metrics

## Reshaping Data

In [2]:
# Extracting only useful columns
df = df[['NDC', 'Manufacturer', 'Product', 'Product Launch Date', 'Estimated LOE Date', 'Major Class', 'Acute/Chronic', 'Prod Form', 'Pack Size', 
         'Pack Quantity', 'Year', 'Month', 'TRx', 'WAC']]

# Dropping all records for which we do not have any price data (WAC)
df = df.dropna(subset=['WAC'])

# Correcting the nan values for TRx (equal to 0)
df.TRx = df.TRx.fillna(0).astype(int)

# Creating a YYYY-MM column to make things easier
df['Date'] = df.Year.astype(str) + '_' + df.Month.astype(str).apply(lambda x: '0' + x if int(x)<10 else x)
df = df[['NDC', 'Manufacturer', 'Product', 'Product Launch Date', 'Estimated LOE Date', 'Major Class', 'Acute/Chronic', 'Prod Form', 'Pack Size', 
         'Pack Quantity', 'Date', 'Year', 'Month', 'TRx', 'WAC']]
print("Shape:", df.shape)

# Removing some duplicates records (we noticed flaws in some entries like syntax)
df = df.drop_duplicates(subset=['NDC', 'Prod Form',
                       'Pack Size', 'Pack Quantity', 'Date'])

df.head()

Shape: (272280, 15)


,NDC,Manufacturer,Product,Product Launch Date,Estimated LOE Date,Major Class,Acute/Chronic,Prod Form,Pack Size,Pack Quantity,Date,Year,Month,TRx,WAC
0,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_10,2014,10,0,244.16
1,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_11,2014,11,6,244.16
2,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_12,2014,12,22,244.16
3,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_01,2015,1,21,244.16
4,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_02,2015,2,15,244.16


## Creating percent changes, previous TRx and cumulated TRx month to month for each drug

In [3]:
# Sorting by drug identifier (NDC) and date
df = df.sort_values(['NDC', 'Date']).reset_index(drop=True)

# Retrieving List of unique drugs 
NDCs = df.NDC.unique()

# Empty Dataframe to store new data
df_pct = pd.DataFrame(columns=df.columns)

# For each drug, compute pct change month to month, lagged TRx and cumulative TRx and append to dataframe
for NDC in tqdm(NDCs):
    # Extracting drug
    df_temp = df[df.NDC == NDC].copy()
    
    # Already sorted by date so we can use pct_change() method
    df_temp['Pct_change'] = df_temp.WAC.pct_change()
    
    # Retrieving volume sold on previous month
    df_temp['Previous_TRx'] = df_temp.TRx.shift(1)
    
    # Computing Cumulative TRx since beginning of year
    df_temp['TRx_cumulative'] = df_temp.groupby(['Year', 'Month']).TRx.sum().groupby(level=0).cumsum().values

    # Appending to dataframe
    df_pct = pd.concat((df_pct, df_temp))

print("Shape: ", df_pct.shape)
df_pct.head(10)

100%|██████████| 4633/4633 [06:22<00:00, 12.10it/s]

Shape:  (242939, 18)


,NDC,Manufacturer,Product,Product Launch Date,Estimated LOE Date,Major Class,Acute/Chronic,Prod Form,Pack Size,Pack Quantity,Date,Year,Month,TRx,WAC,Pct_change,Previous_TRx,TRx_cumulative
0,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_10,2014,10,0,244.16,NaN,NaN,0.0
1,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_11,2014,11,6,244.16,0.00000,0.0,6.0
2,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_12,2014,12,22,244.16,0.00000,6.0,28.0
3,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_01,2015,1,21,244.16,0.00000,22.0,21.0
4,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_02,2015,2,15,244.16,0.00000,21.0,36.0
5,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_03,2015,3,33,244.16,0.00000,15.0,69.0
6,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_04,2015,4,47,244.16,0.00000,33.0,116.0
7,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_05,2015,5,36,265.90,0.08904,47.0,152.0
8,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_06,2015,6,54,265.90,0.00000,36.0,206.0
9,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_07,2015,7,55,265.90,0.00000,54.0,261.0


## Adding boolean: wether or not the price did change from month to month

In [7]:
# Boolean
df_pct['Changed'] = df_pct.Pct_change.apply(lambda x: 1 if (x>0 or x<0) else 0)

# reorganizing columns
df_pct = df_pct[['NDC', 'Manufacturer', 'Product', 'Product Launch Date',
       'Estimated LOE Date', 'Major Class', 'Acute/Chronic', 'Prod Form',
       'Pack Size', 'Pack Quantity', 'Date', 'Year', 'Month', 'TRx', 'Previous_TRx', 
       'TRx_cumulative', 'WAC', 'Pct_change', 'Changed']]

df_pct.head(10)

,NDC,Manufacturer,Product,Product Launch Date,Estimated LOE Date,Major Class,Acute/Chronic,Prod Form,Pack Size,Pack Quantity,Date,Year,Month,TRx,Previous_TRx,TRx_cumulative,WAC,Pct_change,Changed
0,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_10,2014,10,0,NaN,0.0,244.16,NaN,0
1,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_11,2014,11,6,0.0,6.0,244.16,0.00000,0
2,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2014_12,2014,12,22,6.0,28.0,244.16,0.00000,0
3,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_01,2015,1,21,22.0,21.0,244.16,0.00000,0
4,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_02,2015,2,15,21.0,36.0,244.16,0.00000,0
5,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_03,2015,3,33,15.0,69.0,244.16,0.00000,0
6,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_04,2015,4,47,33.0,116.0,244.16,0.00000,0
7,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_05,2015,5,36,47.0,152.0,265.90,0.08904,1
8,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_06,2015,6,54,36.0,206.0,265.90,0.00000,0
9,2143301,Eli Lilly and Co,TRULICITY 10/2014 LLY,10/1/2014 0:00,Sep-26,ANTIDIABETICS,CHRONIC,INJECTABLES,1,0.5,2015_07,2015,7,55,54.0,261.0,265.90,0.00000,0


## Saving Data

In [8]:
# Saving dataset
df_pct.to_csv('drugs_pct_changes_monthly.csv', index=False)